In [19]:
import numpy as np

In [35]:
import unittest

class TestFED(unittest.TestCase):

    def test_fed_tau_by_process_time(self):
        N, tau = fed_tau_by_process_time(500.0, 5, 0.5, True)
        self.assertEquals(N, np.size(tau))
        expected = [0.283858,0.456195,1.327985,60.901379,0.250257,
                    0.298989,0.516557,1.775248,0.252327,0.318043,
                    0.594955,2.517796,0.256534,0.341871,0.698839,
                    3.885568,0.263024,0.371646,0.840030,6.841508,
                    0.272022,0.408991,1.038190,15.288122,]
        self.assertEquals(tau, expected)
        
        
unittest.main(argv=[''], exit=False)

E...
ERROR: test_fed_tau_by_process_time (__main__.TestFED)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-35-af4f65385431>", line 6, in test_fed_tau_by_process_time
    N, tau = fed_tau_by_process_time(500.0, 5, 0.5, True)
  File "<ipython-input-33-b883d473ac91>", line 3, in fed_tau_by_process_time
    N, tau = fed_tau_by_cycle_time(cycle_time, stability, reorder_flag)
  File "<ipython-input-34-2e74bf02d8db>", line 7, in fed_tau_by_cycle_time
    reorder_flag)
TypeError: 'NoneType' object is not iterable

----------------------------------------------------------------------
Ran 4 tests in 0.016s

FAILED (errors=1)


In [32]:
def _fed_tau_internal(num_time_steps, scale, stability, reorder_flag):
    

In [33]:
def fed_tau_by_process_time(stop_time, num_cycles, stability, reorder_flag):
    cycle_time = stop_time/num_cycles
    N, tau = fed_tau_by_cycle_time(cycle_time, stability, reorder_flag)
    return N, tau

In [34]:
def fed_tau_by_cycle_time(stop_time, stability, reorder_flag):
    num_time_steps = int(np.ceil(np.sqrt(3.*stop_time / stability + 0.25)
                                 -0.5 - 1.e-8) + 0.5)
    scale = 3.*stop_time / (stability*float(num_time_steps*(num_time_steps
                                                            +1)))
    N, tau = _fed_tau_internal(num_time_steps, scale, stability, 
                               reorder_flag)
    return N, tau